In [3]:
!pip install nextcord

     -------------------------------------- 927.7/927.7 kB 8.3 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for nextcord: filename=nextcord-2.5.0-py3-none-any.whl size=1021499 sha256=cbdd90efd2d5555a63daec7252d2ec503c19becf4865b41b9a66ad2930b6670c
  Stored in directory: c:\users\djarv\appdata\local\pip\cache\wheels\68\13\46\592f4144d8e5c543971073e4e9a60f933fa36a06b2beecdfeb
Successfully built nextcord


In [1]:
import os
import requests
import json
import time
from utils.SD_gen import generator,upscaling
from PIL import Image
import nextcord 
from nextcord.ext import commands 
from nextcord.ui import Button, View
from io import BytesIO
import base64
#from python_journy import combine_images

In [2]:

def combine_images(images):
    # Create a blank canvas for the final grid image
    images=[Image.open(image) for image in images]
    grid_image = Image.new('RGB', (2 * images[0].width, 2 * images[0].height))

    # Iterate over each image and paste it onto the grid
    for i in range(2):
        for j in range(2):
            # Calculate the coordinates to paste the image
            x = j * images[0].width
            y = i * images[0].height

            # Paste the image onto the grid
            grid_image.paste(images[i*2 + j], (x, y))

    return grid_image

In [3]:
bot =commands.Bot(command_prefix="$", intents=nextcord.Intents.all() )
SD = generator()

In [4]:
results=SD(" highres, masterpirce, blue hair, dark gray skin, 1girl , cat ears, --ar 1:2")

820 410
 highres, masterpirce, blue hair, dark gray skin, 1girl , cat ears, 
API Call
Recived


In [ ]:

class ImageButton(nextcord.ui.Button):
    def __init__(self, image_str: str, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.image_str = image_str

    async def callback(self, interaction: nextcord.Interaction):
        # Get the user who clicked the button
        user = interaction.user
        # Create an embed with the image
        print("button click")
        response=upscaling(self.image_str,SD.API_URL)
        print(response)
        image=response["image"]
        print(image)
        image = BytesIO(base64.decodebytes(image.encode("utf-8"))) 
        image.seek(0)
        file=nextcord.File(image,"image.png")
        await interaction.response.send_message(f"Upscaled", ephemeral=False, file=file)

In [5]:
#Generate
@bot.command()
async def generate(ctx: commands.Context,*, prompt: str):
    #ETA = int(time.time()+80)
    #msg = await ctx.send(f" ETA: <t:{ETA}:R>")
    #print(prompt)
    #results=SD(prompt)
    if not "images" in results:
        print("Error")
        msg=await ctx.send(str(results["detail"]["msg"]))
    else:
        images = results['images']
        
        #view=ImageOptionButton()
        view= View(timeout=None)
        for i,image in enumerate(images):
            button=ImageButton(image_str=image,
                         label=f"U{i+1}", style=nextcord.ButtonStyle.blurple)
            image = BytesIO(base64.decodebytes(image.encode("utf-8"))) 
            view.add_item(button)
            images[i]=image
        grid=combine_images(images)
        
        image_bytes = BytesIO()
        grid.save(image_bytes, format="PNG")
        image_bytes.seek(0)
        #view= ImageGrid_Buttons(images,image_bytes)
        await msg.edit(content="",file=nextcord.File(image_bytes,"image.png"))
        await ctx.send("---",view=view)
       
bot.run(os.getenv("DISCORD_TOKEN"))

RuntimeError: Cannot close a running event loop